In [1]:
from collections import namedtuple

In [24]:
Tag = namedtuple("Tag", ["word", "tag"])    
def readTags(file):
    """
    Creates a list of tagged words from the corpus
    
    Parameter
    String file - dest of file from which sentences are to be read
    
    Return
    sentences - read tags
    """
    tags = []
    sep="\t"
    with open(file, encoding='utf-8') as fp:
        for line in fp:
            line = line.strip()
            if line:
                line = line.split(sep)
                tags.append(Tag(*line))
            else:
                tags.append(Tag("",""))  #append emty tuple to mark sentence ending
    return tags

In [3]:
def tokenize(tags):
    ''' sentence and word tokenization
    '''
    words = []
    entities = []
    
    sentence = []
    entitiesOfSentence = []
    for tag in tags:
        if (tag[0] == "" and tag[1] == ""): 
            words.append(sentence)
            entities.append(entitiesOfSentence)
            sentence = []
            entitiesOfSentence = []
        else:
            sentence.append(tag[0])
            entitiesOfSentence.append(tag[1])
    return words, entities
  
  
def addEntitiyTaggs(posTagged, entities):
    if(len(posTagged) != len(entities)):
        raise ValueError
  
    newTags = []
    sentence = []
    i = 0
    for i in range(len(posTagged)):
        for j in range(len(posTagged[i])):
            sentence.append(((posTagged[i][j][0], posTagged[i][j][1]),entities[i][j]))
        newTags.append(sentence)
        sentence = []
    return newTags

In [27]:
tagsTrain = readTags(r"Data\wnut17train.conll")
print(tagsTrain[0:10])
print(len(tagsTrain))


tagsTest = readTags(r"Data\emerging.test.conll") #error due to encoding 
print(tagsTest[0:10])

wordTaggedSentencesTrain, entitiesTrain = tokenize(tagsTrain)
wordTaggedSentencesTest, entitiesTest = tokenize(tagsTest)
print(wordTaggedSentencesTrain)
print(entitiesTrain)

[Tag(word='@paulwalk', tag='O'), Tag(word='It', tag='O'), Tag(word="'s", tag='O'), Tag(word='the', tag='O'), Tag(word='view', tag='O'), Tag(word='from', tag='O'), Tag(word='where', tag='O'), Tag(word='I', tag='O'), Tag(word="'m", tag='O'), Tag(word='living', tag='O')]
66124
[Tag(word='&', tag='O'), Tag(word='gt', tag='O'), Tag(word=';', tag='O'), Tag(word='*', tag='O'), Tag(word='The', tag='O'), Tag(word='soldier', tag='O'), Tag(word='was', tag='O'), Tag(word='killed', tag='O'), Tag(word='when', tag='O'), Tag(word='another', tag='O')]
[['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.'], ['From', 'Green', 'Newsfeed', ':', 'AHFA', 'extends', 'deadline', 'for', 'Sage', 'Award', 'to', 'Nov', '.', '5', 'http://tinyurl.com/24agj38'], ['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http:

In [28]:
import nltk
#nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')

In [8]:
def posTag(sentences):
    posTaggedSentences = [nltk.pos_tag(sent) for sent in sentences]
    return posTaggedSentences

In [42]:
posTaggedSentencesTrain = posTag(wordTaggedSentencesTrain)
posTaggedSentencesTest = posTag(wordTaggedSentencesTest)
print(posTaggedSentencesTrain[0:10])

completeTaggedSentencesTrain = addEntitiyTaggs(posTaggedSentencesTrain, entitiesTrain)
completeTaggedSentencesTest = addEntitiyTaggs(posTaggedSentencesTest, entitiesTest)
print(completeTaggedSentencesTrain[0:10])

[[('@paulwalk', 'VB'), ('It', 'PRP'), ("'s", 'VBZ'), ('the', 'DT'), ('view', 'NN'), ('from', 'IN'), ('where', 'WRB'), ('I', 'PRP'), ("'m", 'VBP'), ('living', 'VBG'), ('for', 'IN'), ('two', 'CD'), ('weeks', 'NNS'), ('.', '.'), ('Empire', 'NNP'), ('State', 'NNP'), ('Building', 'NNP'), ('=', 'NNP'), ('ESB', 'NNP'), ('.', '.'), ('Pretty', 'NNP'), ('bad', 'JJ'), ('storm', 'NN'), ('here', 'RB'), ('last', 'JJ'), ('evening', 'NN'), ('.', '.')], [('From', 'IN'), ('Green', 'JJ'), ('Newsfeed', 'NN'), (':', ':'), ('AHFA', 'NNP'), ('extends', 'VBZ'), ('deadline', 'NN'), ('for', 'IN'), ('Sage', 'NNP'), ('Award', 'NNP'), ('to', 'TO'), ('Nov', 'NNP'), ('.', '.'), ('5', 'CD'), ('http://tinyurl.com/24agj38', 'NN')], [('Pxleyes', 'NNP'), ('Top', 'NNP'), ('50', 'CD'), ('Photography', 'NNP'), ('Contest', 'NNP'), ('Pictures', 'NNPS'), ('of', 'IN'), ('August', 'NNP'), ('2010', 'CD'), ('...', ':'), ('http://bit.ly/bgCyZ0', 'NN'), ('#photography', 'NN')], [('today', 'NN'), ('is', 'VBZ'), ('my', 'PRP$'), ('last

In [12]:
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI, conlltags2tree, tree2conlltags
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
from nltk.stem import SnowballStemmer
import string

In [30]:
class NamedEntityChunker(ChunkParserI):
    def __init__(self, train_sents, tagger, **kwargs):
        #self.feature_detector = features
        self.tagger = tagger
    def parse(self, sentence):
        chunks = self.tagger.tag(sentence)
        iobTriblets = [(word,pos,entity) for ((word,pos),entity) in chunks]
        return conlltags2tree(iobTriblets)
    

In [37]:
unigramTagger = UnigramTagger(train=completeTaggedSentencesTrain)
bigramTagger = BigramTagger(train=completeTaggedSentencesTrain)
trigramTagger = TrigramTagger(train=completeTaggedSentencesTrain)


In [44]:
nerChunkerUnigram =  NamedEntityChunker(completeTaggedSentencesTrain, unigramTagger)
print(nerChunkerUnigram.evaluate([conlltags2tree([(word, pos, entity) for (word, pos), entity in iobs]) for iobs in completeTaggedSentencesTest]))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("I'm going to Germany this Monday."))))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("Justin is going to France."))))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("Facebook's stock fell by 5% in Frankfurt's stock marked."))))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("The Eiffel Tower is over 300 metres tall"))))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("Liverpool played well at Old Trafford"))))
print(nerChunkerUnigram.parse(nltk.pos_tag(nltk.word_tokenize("Stock in Tesla is soaring"))))

ChunkParse score:
    IOB Accuracy:  93.6%%
    Precision:     16.5%%
    Recall:         2.7%%
    F-Measure:      4.6%%
(S
  I/PRP
  'm/VBP
  going/VBG
  to/TO
  (location Germany/NNP)
  this/DT
  Monday/NNP
  ./.)
(S
  (person Justin/NNP)
  is/VBZ
  going/VBG
  to/TO
  (location France/NNP)
  ./.)
(S
  (corporation Facebook/NNP)
  's/POS
  stock/NN
  fell/VBD
  by/IN
  5/CD
  %/NN
  in/IN
  Frankfurt/NNP
  's/POS
  stock/NN
  marked/VBD
  ./.)
(S
  The/DT
  Eiffel/NNP
  Tower/NNP
  is/VBZ
  over/IN
  300/CD
  metres/NNS
  (location tall/JJ))
(S
  (group Liverpool/NNP)
  played/VBD
  well/RB
  at/IN
  Old/NNP
  Trafford/NNP)
(S Stock/NN in/IN Tesla/NNP is/VBZ soaring/VBG)


In [81]:
def getPosTaggedWordsToSentences(IOTripletsSent):
    return [[(word, pos) for (word, pos), entity in iobs] for iobs in IOTripletsSent]
    


In [89]:
postageedSentences = getPosTaggedWordsToSentences(completeTaggedSentencesTest)

for sent in postageedSentences:
    #print(sent)
    print(tree2conlltags(nerChunkerUnigram.parse(sent)))

[('&', 'CC', 'O'), ('gt', 'NN', 'O'), (';', ':', 'O'), ('*', 'CC', 'O'), ('The', 'DT', 'O'), ('soldier', 'NN', 'O'), ('was', 'VBD', 'O'), ('killed', 'VBN', 'O'), ('when', 'WRB', 'O'), ('another', 'DT', 'O'), ('avalanche', 'NN', 'O'), ('hit', 'VBD', 'O'), ('an', 'DT', 'O'), ('army', 'NN', 'O'), ('barracks', 'NNS', 'O'), ('in', 'IN', 'O'), ('the', 'DT', 'O'), ('northern', 'JJ', 'O'), ('area', 'NN', 'O'), ('of', 'IN', 'O'), ('Sonmarg', 'NNP', 'O'), (',', ',', 'O'), ('said', 'VBD', 'O'), ('a', 'DT', 'O'), ('military', 'JJ', 'O'), ('spokesman', 'NN', 'O'), ('.', '.', 'O')]
[('&', 'CC', 'O'), ('gt', 'NN', 'O'), (';', ':', 'O'), ('*', 'NNP', 'O'), ('Police', 'NNP', 'O'), ('last', 'JJ', 'O'), ('week', 'NN', 'O'), ('evacuated', 'VBD', 'O'), ('80', 'CD', 'O'), ('villagers', 'NNS', 'O'), ('from', 'IN', 'O'), ('Waltengoo', 'NNP', 'O'), ('Nar', 'NNP', 'O'), ('where', 'WRB', 'O'), ('dozens', 'NNS', 'O'), ('were', 'VBD', 'O'), ('killed', 'VBN', 'O'), ('after', 'IN', 'O'), ('a', 'DT', 'O'), ('series',

[('Trio', 'NNP', 'O'), ('@', 'NNP', 'O'), ('phronesismusic', 'JJ', 'O'), ('collaborates', 'NNS', 'O'), ('with', 'IN', 'O'), ('the', 'DT', 'O'), ('Frankfurt', 'NNP', 'O'), ('Radio', 'NNP', 'O'), ('Big', 'NNP', 'O'), ('Band', 'NNP', 'B-group'), ('&', 'CC', 'O'), ('amp', 'NN', 'O'), (';', ':', 'O'), ('@', 'CC', 'O'), ('JulArgJazz', 'NNP', 'O'), ('for', 'IN', 'O'), ('The', 'DT', 'O'), ('Behemoth', 'NNP', 'O'), ('@', 'NNP', 'O'), ('EditionRecords', 'NNP', 'O'), ('…', 'NNP', 'O'), ('https://t.co/UV5UG6ec2w', 'NN', 'O')]
[('@', 'JJ', 'O'), ('realJosephEG', 'NN', 'O'), ('https://t.co/v65QoA0Uyx', 'NN', 'O')]
[('@', 'JJ', 'O'), ('CorporalMum', 'NNP', 'O'), ('YES', 'NNP', 'O'), ('😩', 'NNP', 'O'), ('.', '.', 'O'), ('.', '.', 'O'), ('.', '.', 'O'), ('that', 'IN', 'O'), ("'", "''", 'O'), ('ll', 'JJ', 'O'), ('teach', 'IN', 'O'), ('me', 'PRP', 'O'), ('for', 'IN', 'O'), ('doing', 'VBG', 'O'), ('the', 'DT', 'O'), ('laundry', 'NN', 'O'), ('&', 'CC', 'O'), ('amp', 'NN', 'O'), (';', ':', 'O'), ('playing',

In [39]:
nerChunkerBigram =  NamedEntityChunker(completeTaggedSentencesTrain, bigramTagger)
print(nerChunkerBigram.evaluate([conlltags2tree([(word, pos, entity) for (word, pos), entity in iobs]) for iobs in completeTaggedSentencesTest]))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("I'm going to Germany this Monday."))))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("Justin is going to France."))))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("Facebook's stock fell by 5% in Frankfurt's stock marked."))))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("The Eiffel Tower is over 300 metres tall"))))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("Liverpool played well at Old Trafford"))))
print(nerChunkerBigram.parse(nltk.pos_tag(nltk.word_tokenize("Stock in Tesla is soaring"))))

ChunkParse score:
    IOB Accuracy:  93.8%%
    Precision:     23.1%%
    Recall:         0.3%%
    F-Measure:      0.5%%
(S
  I/PRP
  'm/VBP
  going/VBG
  to/TO
  (location Germany/NNP)
  this/DT
  Monday/NNP
  ./.)
(S
  (person Justin/NNP)
  is/VBZ
  going/VBG
  to/TO
  (location France/NNP)
  ./.)
(S
  (corporation Facebook/NNP)
  's/POS
  stock/NN
  fell/VBD
  by/IN
  5/CD
  %/NN
  in/IN
  Frankfurt/NNP
  's/POS
  stock/NN
  marked/VBD
  ./.)
(S
  The/DT
  Eiffel/NNP
  Tower/NNP
  is/VBZ
  over/IN
  300/CD
  metres/NNS
  tall/JJ)
(S Liverpool/NNP played/VBD well/RB at/IN Old/NNP Trafford/NNP)
(S Stock/NN in/IN Tesla/NNP is/VBZ soaring/VBG)


In [40]:
nerChunkerTrigram =  NamedEntityChunker(completeTaggedSentencesTrain, trigramTagger)
print(nerChunkerTrigram.evaluate([conlltags2tree([(word, pos, entity) for (word, pos), entity in iobs]) for iobs in completeTaggedSentencesTest]))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("I'm going to Germany this Monday."))))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("Justin is going to France."))))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("Facebook's stock fell by 5% in Frankfurt's stock marked."))))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("The Eiffel Tower is over 300 metres tall"))))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("Liverpool played well at Old Trafford"))))
print(nerChunkerTrigram.parse(nltk.pos_tag(nltk.word_tokenize("Stock in Tesla is soaring"))))

ChunkParse score:
    IOB Accuracy:  93.8%%
    Precision:     16.7%%
    Recall:         0.1%%
    F-Measure:      0.2%%
(S
  I/PRP
  'm/VBP
  going/VBG
  to/TO
  (location Germany/NNP)
  this/DT
  Monday/NNP
  ./.)
(S (person Justin/NNP) is/VBZ going/VBG to/TO France/NNP ./.)
(S
  (corporation Facebook/NNP)
  's/POS
  stock/NN
  fell/VBD
  by/IN
  5/CD
  %/NN
  in/IN
  Frankfurt/NNP
  's/POS
  stock/NN
  marked/VBD
  ./.)
(S
  The/DT
  Eiffel/NNP
  Tower/NNP
  is/VBZ
  over/IN
  300/CD
  metres/NNS
  tall/JJ)
(S Liverpool/NNP played/VBD well/RB at/IN Old/NNP Trafford/NNP)
(S Stock/NN in/IN Tesla/NNP is/VBZ soaring/VBG)


In [160]:
import nltk.data, nltk.chunk, nltk.corpus
nltk.download('maxent_ne_chunker')
nltk.download('words')


# load the default chunker for nltk.chunk.ne_chunk
ne_chunker = nltk.data.load(nltk.chunk._MULTICLASS_NE_CHUNKER)
print(ne_chunker.evaluate([conlltags2tree([(word, pos, entity) for (word, pos), entity in iobs]) for iobs in completeTaggedSentencesTest]))



[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
ChunkParse score:
    IOB Accuracy:  91.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
